In [1]:
import enum
import glob
import os
from hashlib import new
from pathlib import Path
import time

import functools

import numpy as np
import pandas as pd
import scipy
from flyingsquid.label_model import LabelModel as LMsquid
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from snorkel.labeling.model import LabelModel as LMsnorkel
from snorkel.labeling.model import MajorityLabelVoter

In [2]:
from sklearn.exceptions import UndefinedMetricWarning

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [8]:
def list2Nested(l, nested_length):
    return [l[i:i+nested_length] for i in range(0, len(l), nested_length)]

In [9]:
# Fetch UMLS ranks

sum_lf_p = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/lf_p_summary_tuipio2_train.csv'
sum_lf_i = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/lf_i_summary_tuipio2_train.csv'
sum_lf_o = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/lf_o_summary_tuipio2_train.csv'


def fetchRank(sum_lf_d):
    
    ranked_umls_coverage = dict()    
    umls_coverage_ = dict()
    
    data=pd.read_csv(sum_lf_d, sep='\t')
    
    for index, row in data.iterrows():
        if row[0].startswith('UMLS_new_fuzzy'):
            umls_coverage_[row[0]] = row[3]
    
    umls_coverage_sorted = sorted(umls_coverage_.items(), key=lambda x: x[1], reverse=True)
    
    for i in umls_coverage_sorted:
        k = str(i[0]).split('_')[-1]
        ranked_umls_coverage[k] = i[1]

    return ranked_umls_coverage

ranksorted_p_umls = fetchRank(sum_lf_p)
ranksorted_i_umls = fetchRank(sum_lf_i)
ranksorted_o_umls = fetchRank(sum_lf_o)

In [10]:
# Partition LF's

def partitionLFs(umls_d):
    
    keys = list(umls_d.keys())

    partitioned_lfs = [ ]
    
    for i in range( 0, len(keys) ):

        if i == 0 or i == len(keys):
            if i == 0:
                partitioned_lfs.append( [keys] )
            if i ==len(keys):
                temp3 = list2Nested(keys, 1)
                partitioned_lfs.append( temp3 )
        else:
            temp1, temp2 = keys[:i] , keys[i:]
            temp3 = list2Nested( keys[:i], 1)
            temp3.append( keys[i:] )
            partitioned_lfs.append( temp3 )
    
    return partitioned_lfs


partitioned_p_umls = partitionLFs(ranksorted_p_umls)
partitioned_i_umls = partitionLFs(ranksorted_i_umls)
partitioned_o_umls = partitionLFs(ranksorted_o_umls)

In [11]:
import LMutils

# validation_labels   
# validation_labels_tui_pio2   
file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/validation_labels_tui_pio2.tsv'
df_data = pd.read_csv(file, sep='\t', header=0)

In [13]:
Y_tokens = df_data['tokens']
df_data_train, df_data_val = train_test_split(df_data, test_size=0.20, shuffle=False)

In [17]:
# Read Candidate labels from multiple LFs
indir = '/mnt/nas2/results/Results/systematicReview/distant_pico/candidate_generation'
pathlist = Path(indir).glob('**/*.tsv')

tokens = []

lfs = dict()

for file in pathlist:

    k = str( file ).split('candidate_generation/')[-1].replace('.tsv', '').replace('/', '_')
    mypath = Path(file)
    if mypath.stat().st_size != 0:
        data = pd.read_csv(file, sep='\t', header=0)
    if len(tokens) == 0:
        tokens.extend( list(data.tokens) )
    
    sab = data.columns[-1]
    if len(list( data[sab] )) == 1354953:
        lfs[str(k)] = list( data[sab] )[:len(Y_tokens)]


print( 'Total number of tokens in validation set: ', len(tokens) )
print( 'Total number of LFs in the dictionary', len(lfs) )

Total number of tokens in validation set:  1354953
Total number of LFs in the dictionary 617


In [18]:
def lf_levels(umls_d:dict, pattern:str, picos:str):

    umls_level = dict()

    for key, value in umls_d.items():   # iter on both keys and values
        search_pattern = pattern + picos
        if key.startswith(search_pattern):
            k = str(key).split('_')[-1]
            umls_level[ k ] = value

    return umls_level

# Level 1: UMLS
umls_p = lf_levels(lfs, 'UMLS_new_fuzzy_', 'p')
umls_i = lf_levels(lfs, 'UMLS_new_fuzzy_', 'i')
umls_o = lf_levels(lfs, 'UMLS_new_fuzzy_', 'o')

# Level 2: non UMLS
nonumls_p = lf_levels(lfs, 'nonUMLS_fuzzy_', 'P')
nonumls_i = lf_levels(lfs, 'nonUMLS_fuzzy_', 'I')
nonumls_o = lf_levels(lfs, 'nonUMLS_fuzzy_', 'O')

# Level 3: DS
ds_p = lf_levels(lfs, 'DS_fuzzy_', 'P')
ds_i = lf_levels(lfs, 'DS_fuzzy_', 'I')
ds_o = lf_levels(lfs, 'DS_fuzzy_', 'O')

# Level 4: dictionary, rules, heuristics
heur_p = lf_levels(lfs, 'heuristics_direct_', 'P')
heur_i = lf_levels(lfs, 'heuristics_direct_', 'I')
heur_o = lf_levels(lfs, 'heuristics_direct_', 'O')

dict_p = lf_levels(lfs, 'dictionary_direct_', 'P')
dict_i = lf_levels(lfs, 'dictionary_direct_', 'I')
dict_o = lf_levels(lfs, 'dictionary_direct_', 'O')

In [19]:
def compare(s, t):
    return sorted(s) == sorted(t)

def getLFs(partition:list, umls_d:dict, seed_len:int):

    all_lfs_combined = []
    
    for lf in partition: # for each lf in a partition
        
        combine_here = [0] * seed_len

        for sab in lf:
            new_a = umls_d[sab]
            old_a = combine_here
            temp_a = []
            for o_a, n_a in zip(old_a, new_a):
                if compare([o_a, n_a] ,[-1, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                elif compare([o_a, n_a] ,[0, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                elif compare([o_a, n_a] ,[-1, 0]) == True:
                    replace_a = min( o_a, n_a )
                    temp_a.append( replace_a )
                else:
                    temp_a.append( o_a )

            combine_here = temp_a

        all_lfs_combined.append( combine_here )

    return all_lfs_combined

In [27]:
def grid_search(model_class,
                model_class_init,
                param_grid,
                train=None,
                dev=None,
                other_train=None,
                n_model_search=5,
                val_metric='f1_macro',
                seed=1234,
                checkpoint_gt_mv=False,
                tag_fmt_ckpnt='IO'):
    
    
    """Simple grid search helper function

    Parameters
    ----------
    model_class
    model_class_init
    param_grid
    train
    dev
    n_model_search
    val_metric
    seed

    Returns
    -------
    

    """
    
    L_train, Y_train = train
    L_dev, Y_dev = dev
    
    # sample configs
    params = sample_param_grid(param_grid, seed)[:n_model_search]
    
    defaults = {'seed': seed}
    best_score, best_config = 0.0, None
    print(f"Grid search over {len(params)} configs")
    
    for i, config in enumerate(params):
        print(f'[{i}] Label Model')
        config = dict(zip(param_grid.keys(), config))
        config.update({param: value for param, value in defaults.items() if param not in config})

        model = model_class(**model_class_init)
        model.fit(L_train, Y_dev, **config)
        
        y_pred = model.predict(L_dev)
        
        if tag_fmt_ckpnt == 'IO':
            y_gold = np.array([0 if y == 0 else 1 for y in Y_dev])
        else:
            y_gold = Y_dev
            
            
        if -1 in y_pred:
            print("Label model predicted -1 (TODO: this happens inconsistently)")
            continue
            
        # use internal label model scorer to score the prediction
        metrics = model.score(L=L_dev,
                              Y=y_gold,
                              metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                              tie_break_policy='random')
        
    
        # compare learned model against MV on same labeled dev set
        # skip if LM less than MV
        if checkpoint_gt_mv:
            mv_metrics = model.score(L=L_dev,
                                  Y=y_gold,
                                  metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                  tie_break_policy='random')

            if metrics[val_metric] < mv_metrics[val_metric]:
                continue
                
        if not best_score or metrics[val_metric] > best_score[val_metric]:
            print(config)
            best_score = metrics
            best_config = config
            
            # print training set score if we have labeled data
            if np.any(Y_train):
                y_pred = model.predict(L_train)

                if tag_fmt_ckpnt == 'IO':
                    y_gold = np.array([0 if y == 0 else 1 for y in Y_train])
                else:
                    y_gold = Y_train

                metrics = model.score(L=L_train,
                                      Y=y_gold,
                                      metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                      tie_break_policy='random')

                print('[TRAIN] {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in metrics.items()])))

            print('[DEV]   {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in best_score.items()])))
            print('-' * 88)
            
            
    # retrain best model
    print('BEST')
    print(best_config)
    model = model_class(**model_class_init)
    
    
    model.fit(L_train, Y_dev, **best_config)
    return model, best_config, best_score

In [28]:
def train(partitioned_d_umls, umls_d, non_umls_d, ds_d, heur_d, dict_d, df_data_train, df_data_val, picos, paramgrid):
   

    best_f1_macro = 0.0
    best_overall_model = ''
    best_overall_config = ''
    
    
    model_class_init = {
        'cardinality': 2, 
        'verbose': True
    }

    num_hyperparams = functools.reduce(lambda x,y:x*y, [len(x) for x in param_grid.values()])
    print("Hyperparamater Search Space:", num_hyperparams)
    n_model_search = num_hyperparams
    


    '''#########################################################################
    # Choosing the number of LF's from UMLS all
    #########################################################################'''
    
    for i, partition in enumerate(partitioned_d_umls):

        combined_lf = getLFs(partition, umls_d, len(Y_tokens))
        assert len(partition) == len(combined_lf)

        print( 'Total number of UMLS partitions: ', len(partition) )
        combined_lf.extend( list(non_umls_d.values()) ) # Combine with level 2
        combined_lf.extend( list(ds_d.values()) ) # Combine with level 3
        combined_lf.extend( list(heur_d.values()) ) # Combine with level 4
        combined_lf.extend( list(dict_d.values()) ) # combine with level 4

        L = np.array(combined_lf)
        L = np.transpose(L)
        L_train, L_val = train_test_split(L, test_size=0.20, shuffle=False)

        Y_train = df_data_train[picos]
        Y_val = df_data_val[picos]
        
        best_model, best_config, best_score = grid_search(LMsnorkel, 
                                           model_class_init, 
                                           paramgrid,
                                           train = (L_train, Y_train),
                                           dev = (L_val, Y_val),
                                           n_model_search=n_model_search, 
                                           val_metric='f1_macro', 
                                           seed=1234,
                                           tag_fmt_ckpnt='IO')
        
        if best_score['f1_macro'] > best_f1_macro:
            best_f1_macro = best_score['f1_macro']
            best_overall_model = best_model
            best_overall_config = best_config
            
        
        print('Best overall macro F1 score: ', best_f1_macro)
        print('Best overall configuration: ', best_overall_config)

In [29]:
param_grid = {
    'lr': [0.001, 0.0001],
    'l2': [0.001, 0.0001],
    'n_epochs': [50, 100, 200, 600, 700, 1000, 2000],
    'prec_init': [0.6, 0.7, 0.8, 0.9],
    'optimizer': ["adamax", "adam", "sgd"],
    'lr_scheduler': ['constant'],
}

In [ ]:
train(partitioned_p_umls, umls_p, nonumls_p, ds_p, heur_p, dict_p, df_data_train, df_data_val, 'p', paramgrid = param_grid)

Hyperparamater Search Space: 336
Total number of UMLS partitions:  1
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.89 | precision: 0.00 | recall: 0.00 | f1: 0.00 | f1_macro: 46.78
[DEV]   accuracy: 87.70 | precision: 0.00 | recall: 0.00 | f1: 0.00 | f1_macro: 46.72
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.05 | precision: 32.17 | recall: 6.29 | f1: 10.52 | f1_macro: 51.77
[DEV]   accuracy: 86.93 | precision: 33.67 | recall: 6.46 | f1: 10.84 | f1_macro: 51.89
----------------------------------------------------------------------------------------
[4] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 600

Total number of UMLS partitions:  2
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.89 | precision: 0.00 | recall: 0.00 | f1: 0.00 | f1_macro: 46.78
[DEV]   accuracy: 87.70 | precision: 0.00 | recall: 0.00 | f1: 0.00 | f1_macro: 46.72
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.84 | precision: 32.06 | recall: 7.77 | f1: 12.51 | f1_macro: 52.70
[DEV]   accuracy: 86.73 | precision: 33.45 | recall: 7.95 | f1: 12.85 | f1_macro: 52.84
----------------------------------------------------------------------------------------
[4] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 600, 'prec_init': 0.6, 'optimizer': 

{'lr': 0.001, 'l2': 0.001, 'n_epochs': 600, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.66 | precision: 26.37 | recall: 19.52 | f1: 22.43 | f1_macro: 56.65
[DEV]   accuracy: 83.66 | precision: 27.27 | recall: 19.73 | f1: 22.90 | f1_macro: 56.88
----------------------------------------------------------------------------------------
[5] Label Model
[6] Label Model
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[

[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
[50] Label Model
[51] Label Model
[52] Label Model
[53] Label Model
[54] Label Model
[55] Label Model
[56] Label Model
[57] Label Model
[58] Label Model
[59] Label Model
[60] Label Model
[61] Label Model
[62] Label Model
[63] Label Model
[64] Label Model
[65] Label Model
[66] Label Model
[67] Label Model
[68] Label Model
[69] Label Model
[70] Label Model
[71] Label Model
[72] Label Model
[73] Label Model
[74] Label Model
[75] Label Model
[76] Label Model
[77] Label Model
[78] Label Model
[79] Label Model
[80] Label Model
[81] Label Mod

[64] Label Model
[65] Label Model
[66] Label Model
[67] Label Model
[68] Label Model
[69] Label Model
[70] Label Model
[71] Label Model
[72] Label Model
[73] Label Model
[74] Label Model
[75] Label Model
[76] Label Model
[77] Label Model
[78] Label Model
[79] Label Model
[80] Label Model
[81] Label Model
[82] Label Model
[83] Label Model
[84] Label Model
[85] Label Model
[86] Label Model
[87] Label Model
[88] Label Model
[89] Label Model
[90] Label Model
[91] Label Model
[92] Label Model
[93] Label Model
[94] Label Model
[95] Label Model
[96] Label Model
[97] Label Model
[98] Label Model
[99] Label Model
[100] Label Model
[101] Label Model
[102] Label Model
[103] Label Model
[104] Label Model
[105] Label Model
[106] Label Model
[107] Label Model
[108] Label Model
[109] Label Model
[110] Label Model
[111] Label Model
[112] Label Model
[113] Label Model
[114] Label Model
[115] Label Model
[116] Label Model
[117] Label Model
[118] Label Model
[119] Label Model
[120] Label Model
[121] Labe

...

[257] Label Model
[258] Label Model
[259] Label Model
[260] Label Model
[261] Label Model
[262] Label Model
[263] Label Model
[264] Label Model
[265] Label Model
[266] Label Model
[267] Label Model
[268] Label Model
[269] Label Model
[270] Label Model
[271] Label Model
[272] Label Model
[273] Label Model
[274] Label Model
[275] Label Model
[276] Label Model
[277] Label Model
[278] Label Model
[279] Label Model
[280] Label Model
[281] Label Model
[282] Label Model
[283] Label Model
[284] Label Model
[285] Label Model
[286] Label Model
[287] Label Model
[288] Label Model
[289] Label Model
[290] Label Model
[291] Label Model
[292] Label Model
[293] Label Model
[294] Label Model
[295] Label Model
[296] Label Model
[297] Label Model
[298] Label Model
[299] Label Model
[300] Label Model
[301] Label Model
[302] Label Model
[303] Label Model
[304] Label Model
[305] Label Model
[306] Label Model
[307] Label Model
[308] Label Model
[309] Label Model
[310] Label Model
[311] Label Model
[312] Labe

[295] Label Model
[296] Label Model
[297] Label Model
[298] Label Model
[299] Label Model
[300] Label Model
[301] Label Model
[302] Label Model
[303] Label Model
[304] Label Model
[305] Label Model
[306] Label Model
[307] Label Model
[308] Label Model
[309] Label Model
[310] Label Model
[311] Label Model
[312] Label Model
[313] Label Model
[314] Label Model
[315] Label Model
[316] Label Model
[317] Label Model
[318] Label Model
[319] Label Model
[320] Label Model
[321] Label Model
[322] Label Model
[323] Label Model
[324] Label Model
[325] Label Model
[326] Label Model
[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Label Model
[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5766100661953499
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_i

[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5775681720927317
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  54
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.22 | precision: 36.46 | recall: 7.50 | f1: 12.45 | f1_macro: 52.78
[DEV]   accuracy: 87.07 | precision: 37.41 | recall: 7.58 | f1: 12.61 | f1_macro: 52.82
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.9, 'optimizer': 'adamax', 'l

[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Label Model
[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.578280466298705
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  55
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.22 | precision: 36.56 | recall: 7.53 | f1: 12.48 | f1_macro: 52.79
[DEV]   accuracy: 87.08 | precision: 37.47 | recall: 7.60 | f1: 12.63 | f1_macro: 52.83
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Mod

[321] Label Model
[322] Label Model
[323] Label Model
[324] Label Model
[325] Label Model
[326] Label Model
[327] Label Model
[328] Label Model
[329] Label Model
[330] Label Model
[331] Label Model
[332] Label Model
[333] Label Model
[334] Label Model
[335] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.578280466298705
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  56
Grid search over 336 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.20 | precision: 36.43 | recall: 7.70 | f1: 12.71 | f1_macro: 52.90
[DEV]   accuracy: 87.06 | precision: 37.44 | recall: 7.81 | f1: 12.92 | f1_macro: 52.96
---------

In [ ]:
train(partitioned_i_umls, umls_i, nonumls_i, ds_i, heur_i, dict_i, df_data_train, df_data_val, 'i', paramgrid = param_grid)

In [ ]:
train(partitioned_o_umls, umls_o, nonumls_o, ds_o, heur_o, dict_o, df_data_train, df_data_val, 'o', paramgrid = param_grid)